# 데이터 불러오기부터

### 가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

#### [참고] AIDU에서 필요한 추가 라이브러리 Import : 아래 코드는 AIDU에서만 실행 가능합니다.

In [ ]:
# from aicentro.session import Session
# from aicentro.framework.tensorflow import Tensorflow as AiduFrm

### 나. 데이터 불러오기

### 만약에 Train/Evaluation 데이터를 따로 전처리 한다면?
* 전처리 기준(특히 정규화 작업 시)이 달라져서 모델의 성능에 악영향을 줄 수 있음

In [ ]:
df = pd.read_csv("onenavi_train.csv",sep="|")
df_eval = pd.read_csv("onenavi_evaluation.csv",sep="|")
df # 데이터 프레임 확인하기

In [ ]:
# 학습/평가 데이터의 전처리 기준을 통일하기위해 데이터 합본
df_total=pd.concat([df,df_eval],ignore_index=True)

#### [참고] AIDU에서는 파일을 불러올 때 아래 예시를 참고 : 아래 코드는 AIDU에서만 실행 가능합니다.

In [ ]:
# # 데이터 셋 로딩
# aicentro_session = Session(verify = False)
# aidu_framework = AiduFrm(session = aicentro_session)
# df = pd.read_csv(aidu_framework.config.data_dir + "/onenavi_train.csv", sep = "|")
# df_eval = pd.read_csv(aidu_framework.config.data_dir + "/onenavi_evaluation.csv", sep = "|")
# # 학습/평가 데이터의 전처리 기준을 통일하기위해 데이터 합본
# df_total=pd.concat([df,df_eval],ignore_index=True)

# 1. 이상치/결측치 처리
## Trash를 판별하는 능력은 좋은 AI모델을 만드는 데 있어 핵심입니다.
+ KeyPoint : 데이터의 형태를 살펴보고 불필요한 데이터를 정제할 수 있다.

[참고] 공식 Document
+ fillna(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)
+ dropna(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

### 가. 결측치 처리

In [ ]:
# 결측치 수 확인하기
df_total.isnull().sum()

##### 우리의 학습 데이터에는 결측치가 없습니다. 원활한 실습을 위해 샘플데이터를 만들어보겠습니다.

In [ ]:
# 샘플테이블 생성
sample = pd.DataFrame(
        {
            'column1':[50,70,np.nan,55],
            'column2':[22,50,66,np.nan]
        })
sample

In [ ]:
# dropna : 제거하기
sample.dropna()

In [ ]:
# fillna(number) : number로 채우기
sample.fillna(0)

In [ ]:
# fillna(method) : 다양한 방법으로 채우기
# 'backfill'/'bfill': 바로 뒤에 값으로 채우기, 'pad'/'ffill': 바로 앞의 값으로 채우기
sample.fillna(method='pad')

### 나. 이상치 처리
#### 이상치(Outlier)는 전적으로 연구자 혹은 개발자가 판단을 해야한다. 자세히 살펴보지않으면 놓치기 쉽상이다.
#### "자세히 보아야 드러난다. 오래 보아야 도움이 된다. Outlier가 그렇다."

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df_total)
plt.show()

In [ ]:
# ET의 분포에서 특이한 점이 있습니다. 약 2만 이상에서 몇개 안보이는 특이치가 있는데요.
#1.5만 이상으로 숫자을 확인해보겠습니다.(1.5만초면 250분)

df_total[df_total['ET']>15000] # 2개

In [ ]:
# Outlier 제거 후 데이터만 남기기

df_total=df_total[df_total['ET']<=15000]
df_total

In [ ]:
# 다시 한 번 확인해 보겠습니다.

sns.pairplot(df_total)
plt.show()

#### 여기서 Tip!
#### 눈에 보이는 것만 가지고 판단하실껀가요?
#### 숨겨진 변수가 있지 않을까요?
#### 평균 시속을 기준으로 분포를 보도록 하겠습니다.

In [ ]:
# 평균시속 변수 만들기 : 속도는 거리 나누기 시간
df_total['PerHour']=(df_total['A_DISTANCE']/1000)/(df_total['ET']/3600)

In [ ]:
# 데이터 분포 확인하기
df_total.describe()

#### 시속이 3500키로?! 이상합니다.

In [ ]:
# 시속 130이상 데이터 수 확인하기 : 몇 개 되지 않으면 제거하자!
len(df_total[df_total['PerHour']>130])

In [ ]:
# Outlier 제거 후 데이터만 남기기
df_total=df_total[df_total['PerHour']<=130]
df_total

In [ ]:
# 다시 한 번 확인해 보겠습니다.

sns.pairplot(df_total)
plt.show()

### 추가데이터를 합치고 확인해보겠습니다.

In [ ]:
df_pnu = pd.read_csv("onenavi_pnu.csv",sep="|") # 주소(시도/시군구 정보)
df_signal = pd.read_csv("onenavi_signal.csv",sep="|") # 경로의 신호등 갯수

In [ ]:
df_total=pd.merge(df_total,df_pnu , on="RID")
df_total=pd.merge(df_total,df_signal , on="RID")
df_total

In [ ]:
# 다시 한 번 확인해 보겠습니다.

sns.pairplot(df_total)
plt.show()

#### [참고] AIDU에서는 파일을 불러올 때 아래 예시를 참고 : 아래 코드는 AIDU에서만 실행 가능합니다.

In [ ]:
# # 추가 데이터 셋 로딩
# df_pnu = pd.read_csv(aidu_framework.config.data_dir + "onenavi_pnu.csv",sep="|") # 주소(시도/시군구 정보)
# df_signal = pd.read_csv(aidu_framework.config.data_dir + "onenavi_signal.csv",sep="|") # 경로의 신호등 갯수
# # 데이터 합치기
# df_total=pd.merge(df_total,df_pnu , on="RID")
# df_total=pd.merge(df_total,df_signal , on="RID")
# df_total

In [ ]:
# 다시 한 번 확인해 보겠습니다.

# sns.pairplot(df_total)
# plt.show()

#### 자, 결측치, 이상치 처리에 있어 우리는 한가지 놓친 것이 있습니다. 데이터 셋 정의를 확인해볼까요?
#### 우리 데이터는 수도권으로 한정되어있다고 정의했었죠?

In [ ]:
df_total['level1_pnu'].unique()

In [ ]:
# 서울/경기/인천으로 데이터 정제
df_total=df_total[(df_total['level1_pnu']=='서울특별시')|(df_total['level1_pnu']=='경기도')|(df_total['level1_pnu']=='인천광역시')]
df_total=df_total.reset_index(drop=True)
df_total

# 3. 더미변수 생성
## 범주형 데이터도 모델링에 활용할 수 있어야 합니다. 이를 위해 더미변수로 변환해주어야 합니다.
+ KeyPoint : 범주형 변수를 더미변수로 변환할 수 있다.

#### 범주형 변수 중에 어떤 변수를 더미로 변환해볼까요?
#### 요일, 시간, 시도 변수를 더미화 하면 좋을 것 같습니다.

In [ ]:
# 진행상황을 체크할 수 있는 라이브러리
!pip install tqdm

In [ ]:
# 요일, 시간 변수 추가
import datetime
from dateutil.parser import parse
from tqdm import tqdm

weekday_list=[]
hour_list=[]
day_list=[]

for w in tqdm(df_total['TIME_DEPARTUREDATE']):
    parse_data_w=parse(w)
    weekday_list.append(parse_data_w.weekday())
    hour_list.append(parse_data_w.hour)
    day_list.append(parse_data_w.day)

df_total['WEEKDAY'] = weekday_list
df_total['HOUR'] = hour_list
df_total['DAY'] = day_list

In [ ]:
# 평가데이터 별도 저장 : 원본 기준(나중에 활용)
new_df_eval=df_total[df_total['DAY']>=27]
new_df_eval.to_csv("onenavi_evaluation_new.csv",sep="|",index=False)

In [ ]:
dummy_fields = ['WEEKDAY','HOUR','level1_pnu','level2_pnu']

for dummy in dummy_fields:
    dummies = pd.get_dummies(df_total[dummy], prefix=dummy, drop_first=False)
    df_total = pd.concat([df_total, dummies], axis=1)

df_total = df_total.drop(dummy_fields,axis=1)

# 4. 데이터 스케일링
##  Feature들의 크기, 범주를 정규화하는 과정을 통해 특정변수의 영향도를 조정해줄 수 있습니다.
+ KeyPoint : 데이터 스케일링을 통해 AI모델 학습에 도움을 줄 수 있다.

In [ ]:
# 기준일자 저장 : why? 날짜가 스케일링이 되면 안됩니다.
data_day=df_total['DAY']

In [ ]:
train_data=df_total.drop(['RID','TIME_DEPARTUREDATE','TIME_ARRIVEDATE','ET','ETAA','PerHour','DAY'],axis=1)
columnNames=train_data.columns
train_data

In [ ]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
feature = pd.DataFrame(scaler.fit_transform(train_data))
feature.columns=columnNames
feature

In [ ]:
# 다시 날짜를 합칠께요 : Train/Evaluation 분리를 위해
feature['DAY']=data_day
# traindata 지정
train_feature=feature[feature['DAY']<=24]
train_feature=train_feature.drop(['DAY'],axis=1)
eval_feature=feature[feature['DAY']>=27]
eval_feature=eval_feature.drop(['DAY'],axis=1)

In [ ]:
len(feature),len(train_feature),len(eval_feature)

In [ ]:
# target 저장
train_target = df_total[df_total['DAY']<=24]['ET']
train_target

In [ ]:
# CSV로 저장
train_feature.to_csv('onenavi_train_feature.csv',index = False,sep='|')
train_target.to_csv('onenavi_train_target.csv',index = False,sep='|')
eval_feature.to_csv('onenavi_eval_feature.csv',index = False,sep='|')